In [ ]:
!pip install ortools
!pip install folium

In [2]:
import random
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model

In [3]:
data = {
    'Nombre': [
        'Hospital Nacional Hipólito Unanue',
        'Hospital Cayetano Heredia',
        'Hospital Santa Rosa',
        'Hospital Nacional Guillermo Almenara Irigoyen',
        'Hospital Nacional Edgardo Rebagliati Martins',
        'Clínica Ricardo Palma - Javier Prado Este',
        'Clínica San Pablo - Sede Surco',
        'Hospital Nacional Dos de Mayo',
        'Hospital Nacional Sergio E. Bernales',
        'Clínica Anglo Americana'
    ]
}

df = pd.DataFrame(data)

In [4]:
lats = [-12.0411954 ,-12.0216057 ,25.797809 ,38.176505 ,-12.05963535 ,-12.0782058 ,-11.96637165 ,-12.055924 ,-11.9141137 ,-12.07442605]
longs =  [-76.99263863265921 ,-77.05509942443221 ,-100.5616261 ,-0.7983121 ,-77.02236228077756 ,-77.0399864964742 ,-77.05641452313083 ,-77.01569082772036 ,-77.03751289728125 ,-76.95597427784205]
camas = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100]

df['Latitud'] = lats
df['Longitud'] = longs
df['Camas'] = camas

In [5]:
def find_corners(latitudes, longitudes):
    min_lat = min(latitudes)
    max_lat = max(latitudes)
    min_lon = min(longitudes)
    max_lon = max(longitudes)

    corners = [
        (min_lat, min_lon),
        (min_lat, max_lon),
        (max_lat, max_lon),
        (max_lat, min_lon)
    ]

    return corners

corners = find_corners(lats, longs)
print("Corner points:", corners)

Corner points: [(-12.0782058, -100.5616261), (-12.0782058, -0.7983121), (38.176505, -0.7983121), (38.176505, -100.5616261)]


In [14]:
def generate_cases(pos1, pos2, n):
    cases = []

    # Conseguimos la latitud y longitud de 2 puntos delimitorios del área
    lat1, lon1 = pos1
    lat2, lon2 = pos2

    # Definimos el rango de la latitud y longitud
    lat_range = max(lat1, lat2) - min(lat1, lat2)
    lon_range = max(lon1, lon2) - min(lon1, lon2)

    for _ in range(n):
        lat = random.uniform(min(lat1, lat2), max(lat1, lat2))
        lon = random.uniform(min(lon1, lon2), max(lon1, lon2))

        # Definimos una severidad entre 1 y 4
        severity = random.randint(1, 4)
        cases.append({'Latitud': lat, 'Longitud': lon, 'Severidad': severity})
    return cases

In [15]:
pos1 = (-11.894451, -76.967502)  # Bottom-left corner
pos2 = (-12.097977, -77.081000)  # Top-right corner

n = 1000                                  # Cantidad de casos
cases = generate_cases(pos1, pos2, n)
patients = pd.DataFrame(cases)

In [8]:
def solve_hospital_assignment(hosp_data, pat_data):
    nHosp = len(hosp_data)
    nPats = len(pat_data)
    max_beds = hosp_data['Camas'].tolist()
    hospital_coords = list(zip(hosp_data['Latitud'], hosp_data['Longitud']))
    patient_coords = list(zip(pat_data['Latitud'], pat_data['Longitud']))
    severities = pat_data['Severidad'].tolist()

    model = cp_model.CpModel()

    # Decision variables
    assignment = np.array([[model.NewBoolVar(f'x_{i}_{j}') for j in range(nPats)] for i in range(nHosp)])

    # Constraints
    for i in range(nHosp):
        model.Add(sum(assignment[i]) <= max_beds[i])  # Each hospital has beds constraint

    for j in range(nPats):
        model.Add(sum(assignment[:, j]) == 1)  # Each patient is assigned to exactly one bed

    # Objective function
    total_distance = sum(
    sum(assignment[i][j] * np.linalg.norm(np.array(hospital_coords[i]) - np.array(patient_coords[j])) for j in range(nPats))
    for i in range(nHosp)
    )


    total_severity = sum(
        cp_model.LinearExpr.WeightedSum(assignment[i], [severities[j] for j in range(nPats)])
        for i in range(nHosp)
    )

    model.Minimize(total_distance + total_severity)

    # Solve the problem
    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        assignments = {
            f'Hospital_{i}': [j for j in range(nPats) if solver.Value(assignment[i, j])]
            for i in range(nHosp)
        }
        return assignments
    else:
        return None

In [20]:
solution = solve_hospital_assignment(df, patients)

In [19]:
import folium
import random

# Crear mapa centrado en la primera ubicación de los hospitales
mapa = folium.Map(location=[lats[0], longs[0]], zoom_start=12)

# Iterar sobre las listas de latitudes y longitudes de los hospitales y agregar marcadores al mapa
for lat, long in zip(lats, longs):
    folium.Marker(location=[lat, long], popup='Hospital', icon=folium.Icon(color='blue', icon='hospital')).add_to(mapa)

# Generar 1000 ubicaciones aleatorias con severidades y agregar marcadores al mapa
for i in range(n):
    latitud = cases[i]['Latitud']
    longitud = cases[i]['Longitud']
    severidad = cases[i]['Severidad']

    if severidad == 1:
        color = 'green'
    elif severidad == 2:
        color = 'yellow'
    elif severidad == 3:
        color = 'orange'
    else:
        color = 'red'

    # Agregar marcador de paciente al mapa con el color correspondiente
    folium.Marker(location=[latitud, longitud], icon=folium.Icon(color=color, icon_size=(2, 2)), popup=f'Severidad: {severidad}').add_to(mapa)

# Guardar el mapa como un archivo HTML
mapa.save('mapa_hospitales_y_severidades.html')


<ipython-input-19-8a57d697e342>:27: UserWarning: color argument of Icon should be one of: {'darkpurple', 'purple', 'pink', 'lightgray', 'red', 'darkred', 'darkblue', 'black', 'lightblue', 'gray', 'blue', 'lightred', 'green', 'lightgreen', 'darkgreen', 'beige', 'white', 'cadetblue', 'orange'}.
  folium.Marker(location=[latitud, longitud], icon=folium.Icon(color=color, icon_size=(2, 2)), popup=f'Severidad: {severidad}').add_to(mapa)
